In [1]:
import pandas as pd
import re

# Countries

In [2]:
countries_df = pd.read_csv("./countries.csv")

In [3]:
def find_country(val):
    # manual match
    if val in ["Australia"]:
        val = "Australian"
    if val in ["British Virgin Islands"]:
        val = "Virgin Islands (British)"
    if val in ["Cabo Verde"]:
        val = "Cape Verde"
    if val in ["Central African Rep."]:
        val = "Central African Republic"
    if val in ["Cote d'Ivoire", "Côte d'Ivoire"]:
        val = "Ivory Coast"
    if val in ["Congo, Dem. Rep.", "Congo, Dem. Rep. of the"]:
        val = "Congo (Kinshasa)"
    if val in ["Congo, Rep.", "Congo, Rep. of"]:
        val = "Congo-Brazzaville"
    if val in ["Egypt, Arab Rep."]:
        val = "Egypt"
    if val in ["Croatia, Rep. of"]:
        val = "Croatia (Hrvatska)"
    if val in ["Czech Rep."]:
        val = "Czech Republic"
    if val in ["Dominican Rep."]:
        val = "Dominican Republic"
    if val in ["Iran, Islamic Rep. of"]:
        val = "Iran (Islamic Republic of)"
    if val in ["Korea, Rep. of"]:
        val = "Korea South"
    if val in ["Kyrgyz Rep."]:
        val = "Kyrgyzstan"
    if val in ["Libya"]:
        val = "Libyan Arab Jamahiriya"
    if val in ["Eswatini, Kingdom of"]:
        val = "Swaziland"
    if val in ["Lao People's Dem. Rep."]:
        val = "Lao PeopleÕs Democratic Republic"
    if val in ["Micronesia, Federated States of"]:
        val = "Micronesia Federated States of"
    if val in ["Moldova, Rep. of"]:
        val = "Moldova Republic of"
    if val in ["North Macedonia, Republic of"]:
        val = "Macedonia"
    if val in ["São Tomé and Príncipe, Dem. Rep. of"]:
        val = "Sao Tome and Principe"
    if val in ["Sint Maarten, Kingdom of the Netherlands"]:
        val = "Saint Martin (French part)"
    if val in ["Slovak Rep."]:
        val = "Slovakia (Slovak Republic)"
    if val in ["St. Lucia"]:
        val = "Saint Lucia"
    if val in ["St. Kitts and Nevis"]:
        val = "Saint Kitts"
    if val in ["St. Vincent and the Grenadines"]:
        val = "Saint Vincent Grenadines"
    if val in ["Syrian Arab Rep."]:
        val = "Syrian Arab Republic"
    if val in ["Timor-Leste, Dem. Rep. of"]:
        val = "East Timor"
    if val in ["China, P.R.: Hong Kong"]:
        val = "Hong Kong"
    if val in ["China, P.R.: Macao"]:
        val = "Macao S.A.R."
    if val in ["Curaçao, Kingdom of the Netherlands"]:
        val = "Curaçao"
    if val in ["Kosovo, Rep. of"]:
        val = "Kosovo"
    if val in ["South Sudan, Rep. of"]:
        val = "South Sudan"
    if val in ["West Bank and Gaza"]:
        val = "Palestinian Territory"

    # EOL manual match
    find_country = countries_df[countries_df["country"].str.lower() == str(val).lower()]
    if find_country.empty:
        # Another custom way
        val_lower = str(val).lower()
        val_tmp = val_lower.lower().split(',')
        for index, row in countries_df.iterrows():
            # Convert the country name in the DataFrame to lowercase
            country_name = row['country'].lower()
            if val_lower == 'nan':
                return None
            if country_name not in val_tmp:
                continue
            return row["id"]
        # EOL Another custom way

        print("404", val)
        return val
    return find_country["id"].to_list()[0]

# CREATE REGIONS

In [4]:
df = pd.read_excel("../lib_dataset.xlsx", sheet_name=0, header=4)

In [5]:
df = df.rename(columns={"Country": "country", "Region/area.1": "region"})

In [6]:
regions = df[["country", "region"]]

In [7]:
regions = regions[regions["region"].notna()]

In [8]:
regions['country'] = regions['country'].str.replace('\n', ' ').str.strip()
regions['region'] = regions['region'].str.replace('\n', ' ').str.strip()
regions['region'] = regions['region'].str.replace(r'^\s+|\s+?$', '', regex=True)

In [9]:
split_df = regions.assign(region=regions['region'].str.split(' / ')).explode('region')
split_df = regions.assign(region=regions['region'].str.split('/ ')).explode('region')
split_df = regions.assign(region=regions['region'].str.split(' /')).explode('region')
split_df = regions.assign(region=regions['region'].str.split('/')).explode('region')
split_df = split_df.drop_duplicates(subset=['country', 'region'])
split_df.reset_index(drop=True, inplace=True)

## Map regions with countries

In [10]:
split_df["country_id"] = split_df["country"].apply(find_country)

In [11]:
split_df.head()

,country,region,country_id
0,Argentina,Non-Metropolitan Urban,11
1,Argentina,All,11
2,Argentina,Rural,11
3,Bangladesh,Urban,19
4,Bangladesh,Satellite Cities,19


## Export Regions to CSV

In [12]:
# group_df = split_df.groupby('region')['country'].apply(lambda x: list(set(x))).reset_index()
group_df = split_df.groupby('region').agg({
    "country": "unique",
    "country_id": "unique"
}).reset_index()

In [13]:
group_df = group_df[group_df['region'].str.strip() != '']

In [14]:
group_df['region'] = group_df['region'].str.replace(r'^\s+|\s+?$', '', regex=True)

In [15]:
group_df['country_id'] = group_df['country_id'].apply(list)
group_df['country'] = group_df['country'].apply(list)

In [16]:
group_df["id"] = group_df.reset_index().index + 1

In [17]:
new_column_order = ["id", "region", "country_id", "country"]

In [18]:
group_df = group_df[new_column_order]

In [19]:
group_df.head()

,id,region,country_id,country
1,1,All,"[11, 23, 35, 37, 38, 43, 44, 109, 50, 65, 66, ...","[Argentina, Belize, Burkina Faso, Cambodia, Ca..."
2,2,All coffee and cocoa growing regions,[176],[Peru]
3,3,Andhra Pradesh,[101],[India]
4,4,Baja California,[145],[Mexico]
5,5,Belém and the metropolitan area,[31],[Brazil]


In [20]:
group_df.to_csv('regions.csv', index=False)

In [21]:
regions = group_df

# CREATE LIVING INCOME BENCHMARK

source (kolom source information (K))
region (region)
year (source information (L))

household_size (kolom Average household Size #people)

LCU, USD, EUR

In [22]:
df2 = pd.read_excel("../lib_dataset.xlsx", sheet_name=0, header=6)

In [23]:
country = df2.iloc[:, 0]
region = df2.iloc[:, 3]
adults = df2.iloc[:, 7]
household_size = df2.iloc[:, 6]
source = df2.iloc[:, 11] #10
year = df2.iloc[:, 12] #11
lcu = df2.iloc[:, 14] #13
usd = df2.iloc[:, 15] #14
eur = df2.iloc[:, 16] #15

In [24]:
lib_tmp = {
    'country': country,
    'region': region,
    'adults': adults,
    'household_size': household_size,
    'source': source,
    'year': year,
    'LCU': lcu,
    'USD': usd,
    'EUR': eur,
}

In [25]:
lib = pd.DataFrame(lib_tmp)

In [26]:
lib['country'] = lib['country'].str.replace('\n', ' ').str.strip()
lib['region'] = lib['region'].str.replace('\n', ' ').str.strip()
lib['region'] = lib['region'].str.replace(r'^\s+|\s+?$', '', regex=True)

In [27]:
lib_split = lib.assign(region=lib['region'].str.split(' / ')).explode('region')
lib_split = lib.assign(region=lib['region'].str.split('/ ')).explode('region')
lib_split = lib.assign(region=lib['region'].str.split(' /')).explode('region')
lib_split = lib.assign(region=lib['region'].str.split('/')).explode('region')
lib_split = lib_split.drop_duplicates(subset=['country', 'region'])
lib_split.reset_index(drop=True, inplace=True)

## Map benchmark with countries and regions

In [28]:
def find_region(val):
    find_region = regions[regions["region"].str.lower() == str(val).lower()]
    if find_region.empty:
        print("404", val)
        return val
    return int(find_region["id"].to_list()[0])

In [29]:
lib_split["country_id"] = lib_split["country"].apply(find_country)

In [30]:
lib_split['region'] = lib_split['region'].str.replace(r'^\s+|\s+?$', '', regex=True)

In [31]:
lib_split["region_id"] = lib_split["region"].apply(find_region)

404 
404 nan


## Export benchmark to CSV

In [32]:
def calculate_average_household_size(x):
    children = (x['household_size'] - x['adults']) * 0.3
    adults = x['adults']
    if adults > 1:
        adults = 1 + (0.5 * (adults - 1))
    return adults + children

In [33]:
lib_split = lib_split[lib_split["region"].notna()]

In [34]:
lib_split['region_id'] = pd.to_numeric(lib_split['region_id'], errors='coerce')
lib_split['region_id'] = lib_split['region_id'].fillna(0).astype(int)

In [35]:
lib_split['year'] = lib_split['year'].fillna(0).astype(int)

In [36]:
lib_split['household_size'] = lib_split['household_size'].fillna(0).astype(int)
lib_split['adults'] = lib_split['adults'].fillna(0).astype(int)

In [37]:
lib_split['household_size'] = lib_split[['household_size','adults']].apply(calculate_average_household_size, axis=1)

In [38]:
lib_split = lib_split[lib_split['region'].str.strip() != '']

In [39]:
lib_split["id"] = lib_split.reset_index().index + 1

In [40]:
new_column_order = ['id', 'country_id', 'region_id', 'country', 'region', 'household_size', 'source', 'year', 'LCU', 'USD', 'EUR']

In [41]:
lib_split = lib_split[new_column_order]

In [42]:
lib_split.head()

,id,country_id,region_id,country,region,household_size,source,year,LCU,USD,EUR
0,1,11.0,68,Argentina,Non-Metropolitan Urban,2.1,https://www.globallivingwage.org/living-wage-r...,2022,37180.68,284.655199,270.315338
1,2,11.0,1,Argentina,All,2.1,https://www.globallivingwage.org/living-wage-r...,2022,37180.68,284.655199,270.315338
2,3,11.0,74,Argentina,Rural,2.1,https://globallivingwage.org/living-income-ref...,2022,30800.75,235.810470,223.931223
3,4,19.0,79,Bangladesh,Urban,2.1,https://www.globallivingwage.org/living-wage-b...,2022,7602.00,82.859691,78.685531
4,5,19.0,43,Bangladesh,Satellite Cities,2.1,https://www.globallivingwage.org/living-wage-b...,2022,7602.00,82.859691,78.685531


In [43]:
lib_split.to_csv('li_benchmark.csv', index=False)

# CPI

In [44]:
df3 = pd.read_excel("../lib_dataset.xlsx", sheet_name=3, header=2)

In [45]:
# remove unnmaed column
df3 = df3.drop(columns=["Unnamed: 11"])

In [46]:
df3 = df3.rename(columns={"Country Name": "country"})

## Map with Countries

In [47]:
df3["country_id"] = df3["country"].apply(find_country)

## Transform CPI table

In [48]:
# transform int country, country_id, year, value format
melted_df = pd.melt(df3, id_vars=['country', 'country_id'], var_name='year', value_name='value')
melted_df['value'] = melted_df['value'].fillna(0)

In [49]:
melted_df.reset_index(drop=True, inplace=True)
melted_df["id"] = melted_df.reset_index().index + 1

In [50]:
new_column_order = ["id", "country", "country_id", "year", "value"]

In [51]:
cpi_df = melted_df[new_column_order]

## Export CPI to CSV

In [52]:
cpi_df.head()

,id,country,country_id,year,value
0,1,"Afghanistan, Islamic Rep. of",1,2012,90.717067
1,2,Albania,3,2012,87.567588
2,3,Algeria,4,2012,155.054167
3,4,Angola,7,2012,24.375141
4,5,Anguilla,8,2012,107.285000


In [53]:
cpi_df.to_csv('cpi.csv', index=False)